# 101 Asynchronous programming with Python
## Module 1

### Agenda:

* Key definitions
  * Concurrency, multitasking, 
  * Coroutines
* The Python syntax: what means "async" and why you need "await"
* What is event loop and why you should not care about it.


# Key devinitions

## Concurrency and parallelism

**Concurrent computing**

<div align="center"><img src="../images/whooper%20swans%20on%20snow.jpg?raw=1" alt="whooper swans on snow" width="200"/></div>

is a form of computing in which several computations are executed concurrently—during overlapping time periods—instead of sequentially, with one completing before the next starts.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Concurrent_computing">Wikipedia / Concurrent computing </a>
</div>

---



**Parallelism (parallel computing)**

<div align="center"><img src="../images/three%20seagulls.jpg?raw=1" alt="three seagulls" width="200"/></div>

is a simultaneous execution of different parts of a program on multiple processors.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Parallel_computing">Wikipedia / Parallel computing </a>
</div>

👉 *You may have parallelism without concurrency, and concurrency without parallelism.* 👈

---

## Multiprocessing, multithreading, multitasking
**(in Python)**

**Multiprocessing**

<div align="center"><img src="../images/seagulls%20in%20a%20row.jpg" alt="seagulls in a row" width="200"/></div>

The load is distributed between multiple OS processes that have an
ability to run in parallel on different CPUs.  It's good for heavy
computation tasks, and allows to achieve a real parallelism.

Still, spawning processes is expensive, and they may be hard to manage
and communicate.  *If things go wrong, you may end up with
orphan processes or an army of zombies.*

---
**Multithreading**
<div align="center"><img src="../images/geese.jpg" alt="geese" width="200"/></div>

Threads are "execution units" of a process, managed by OS.  They are
more easy to create.  Since threads share resources, you can convert
your single-thread code to a multithreaded one with just a bit of
changes.

On the other hand, sharing resources and OS-managed-context-switching
means that you need to pay attention to tread safety.  It also
add complications to signal handling (like raising KeyboardInterrupt).

Threads are expensive, still not so expensive as processes.
In Python they allow to execute the code concurrently,
but not in parallel.

---

**Multitasking**
<div align="center"><img src="../images/four%20rock%20doves%20on%20gray%20floor.jpg" alt="four rock doves on gray floor" width="200"/></div>

Allows to execute concurrent tasks that share common computation
resources, such as CPU and RAM.

With **cooperative multitasking**, the process (the program) decides
itself when to interrupt its execution and yield control to
other processes.

Cooperative multitasking allows simpler implementations, since there is
no implicit context switching and no thread safety is needed.
It allows to have a high number of tasks executed concurrently.

Still, creating applications with cooperative multitasking require
care to make sure each task fairly use the computation resources
and timely yields control.  Also a high number of tasks still
can affect the application performance.

---
